In [49]:
import numpy as np
import pandas as pd
from scipy.stats import norm
from statsmodels.sandbox.stats.multicomp import multipletests
import joblib
import time
from joblib import Parallel, delayed
import multiprocessing
from random import randint
import rpy2
import rpy2.robjects as robjects
#from rpy2.robjects import pandas2ri
from rpy2.robjects.vectors import FloatVector
# Load the required R package
robjects.r('''
library(elitism)
''')
## Calculate Score Quantile
def scores_ord_bayes(P_sort,theta,th):
    dens_sort = np.exp((norm.ppf(P_sort) * theta) - (0.5 * (theta ** 2)));
    K = np.size(P_sort);
    score_mat = np.zeros((K-1,K-1));
    score_mat[0,:] = ((np.delete(dens_sort,K-1)*(np.delete(P_sort,K-1)<alpha)*(1+dens_sort[K-1]))+((1+np.delete(dens_sort,K-1))*(P_sort[K-1]<alpha)*dens_sort[K-1]))/4
    for j in 1+np.arange(K-2):
        score_mat[j,0:K-j-1] = (((score_mat[j-1,0:K-j-1]>th[j-1])*dens_sort[0:K-j-1]*np.prod(1+dens_sort[K-j-1:K]))/(2**(j+2)))+(((1+dens_sort[0:K-j-1])/2)*score_mat[j-1,K-j-1]*(score_mat[j-1,K-j-1]>th[j-1]))
    return(score_mat[K-2,0])
## Calculate Decision
def decision_bayes(P,theta,th):
    P_sort = np.sort(P);
    dens_sort = np.exp((norm.ppf(P_sort) * theta) - (0.5 * (theta ** 2)));
    K = np.size(P_sort);
    score_mat = np.zeros((K-1,K-1));
    score_mat[0,:] = ((np.delete(dens_sort,K-1)*(np.delete(P_sort,K-1)<alpha)*(1+dens_sort[K-1]))+((1+np.delete(dens_sort,K-1))*(P_sort[K-1]<alpha)*dens_sort[K-1]))/4
    for j in 1+np.arange(K-2):
        score_mat[j,0:K-j-1] = (((score_mat[j-1,0:K-j-1]>th[j-1])*dens_sort[0:K-j-1]*np.prod(1+dens_sort[K-j-1:K]))/(2**(j+2)))+(((1+dens_sort[0:K-j-1])/2)*score_mat[j-1,K-j-1]*(score_mat[j-1,K-j-1]>th[j-1]));
    D_sort= np.append(1*(np.rot90(score_mat).diagonal()>th)[::-1],int(P_sort[K-1]<alpha));
    D_sort = np.cumprod(D_sort);
    return(D_sort[np.argsort(np.argsort(P))])
## Calculate Score Quantile
def scores_ord_pi1(P_sort,theta,th):
    dens_sort = np.exp((norm.ppf(P_sort) * theta) - (0.5 * (theta ** 2)));
    K = np.size(P_sort);
    score_mat = np.zeros((K-1,K-1));
    score_mat[0,:] = ((np.delete(dens_sort,K-1)*(np.delete(P_sort,K-1)<alpha))+((P_sort[K-1]<alpha)*dens_sort[K-1]))
    for j in 1+np.arange(K-2):
        score_mat[j,0:K-j-1] = ((score_mat[j-1,0:K-j-1]>th[j-1])*dens_sort[0:K-j-1])+(score_mat[j-1,K-j-1]*(score_mat[j-1,K-j-1]>th[j-1]))
    return(score_mat[K-2,0])
## Calculate Decision
def decision_pi1(P,theta,th):
    P_sort = np.sort(P);
    dens_sort = np.exp((norm.ppf(P_sort) * theta) - (0.5 * (theta ** 2)));
    K = np.size(P_sort);
    score_mat = np.zeros((K-1,K-1));
    score_mat[0,:] = ((np.delete(dens_sort,K-1)*(np.delete(P_sort,K-1)<alpha))+((P_sort[K-1]<alpha)*dens_sort[K-1]))
    for j in 1+np.arange(K-2):
        score_mat[j,0:K-j-1] = ((score_mat[j-1,0:K-j-1]>th[j-1])*dens_sort[0:K-j-1])+(score_mat[j-1,K-j-1]*(score_mat[j-1,K-j-1]>th[j-1]))
    D_sort= np.append(1*(np.rot90(score_mat).diagonal()>th)[::-1],int(P_sort[K-1]<alpha));
    D_sort = np.cumprod(D_sort);
    return(D_sort[np.argsort(np.argsort(P))])
## Calculate Last Step Score Quantile
def scores_hom_bayes(P,theta):
    P_sort = np.sort(P);
    _, p_values, _, _ = multipletests(np.delete(P_sort,0), alpha=alpha, method='hommel')
    score = np.sum(np.append((multipletests(np.delete(P_sort,1), alpha=alpha, method='hommel')[0]*1)[0],1*(p_values < alpha))*(np.apply_along_axis(np.prod,1,-np.eye(np.size(P_sort))+1+np.exp((norm.ppf(P_sort) * theta) - (0.5 * (theta ** 2))))/(2**np.size(P)))); #pi_bayes
    return(score)
# Calculate Dcision
def decisions_hom_bayes(P,theta,th):
#     pass;
    P_sort = np.sort(P);
    _, p_values, _, _ = multipletests(np.delete(P_sort,0), alpha=alpha, method='hommel')
    score_ind = np.sum(np.append((multipletests(np.delete(P_sort,1), alpha=alpha, method='hommel')[0]*1)[0],1*(p_values < alpha))*(np.apply_along_axis(np.prod,1,-np.eye(np.size(P_sort))+1+np.exp((norm.ppf(P_sort) * theta) - (0.5 * (theta ** 2))))/(2**np.size(P)))); #pi_bayes
    decisions = np.append((multipletests(np.delete(P_sort,1), alpha=alpha, method='hommel')[0]*1)[0],1*(p_values < alpha))*1*(score_ind>th)
    return(decisions[np.argsort(np.argsort(P))])
## Calculate Last Step Score Quantile
def scores_hom_pi1(P,theta):
    P_sort = np.sort(P);
    _, p_values, _, _ = multipletests(np.delete(P_sort,0), alpha=alpha, method='hommel')
    score = np.sum(np.append((multipletests(np.delete(P_sort,1), alpha=alpha, method='hommel')[0]*1)[0],1*(p_values < alpha))*(np.exp((norm.ppf(P_sort) * theta) - (0.5 * (theta ** 2))))); #pi_1
    return(score)
# Calculate Dcision
def decisions_hom_pi1(P,theta,th):
#     pass;
    P_sort = np.sort(P);
    _, p_values, _, _ = multipletests(np.delete(P_sort,0), alpha=alpha, method='hommel')
    score_ind = np.sum(np.append((multipletests(np.delete(P_sort,1), alpha=alpha, method='hommel')[0]*1)[0],1*(p_values < alpha))*(np.exp((norm.ppf(P_sort) * theta) - (0.5 * (theta ** 2))))); #pi_1
    decisions = np.append((multipletests(np.delete(P_sort,1), alpha=alpha, method='hommel')[0]*1)[0],1*(p_values < alpha))*1*(score_ind>th)
    return(decisions[np.argsort(np.argsort(P))])

def GouConsonantRej(p_values,level):
    # Convert NumPy array to an R numeric vector
    p_values_r = robjects.FloatVector(p_values)
    
    # Assign to R environment
    robjects.r.assign("p_values", p_values_r)
    
    # Run the R function and capture the output
    mtp_output = robjects.r('''
    mtp_results <- elitism::p.adjust(p_values, method = "gtxr")
    mtp_results  # Return the result back to Python
    ''')
    return(1*(np.array(mtp_output)<level))


## Upload the thresholds for the effect size $\theta$ which give target Bonferroni power of $0.3$. 

In [309]:
################# Thresholds #################
x = np.load('thresholds_pow_30_5.npy',allow_pickle=True)
th_bayes = x[2]
th_pi1 = x[3]
thhom_bayes = x[0]
thhom_pi1 = x[1]
alpha=0.05
K=5
theta = norm.ppf(alpha/K)-norm.ppf(0.3)

## Upload the $p$-values. We have $5$ $p$-values in each of the $248$ studies.

In [312]:
df = pd.read_csv("pvmat.csv")                        # add options: sep=",", header=0, dtype=...
df = df.drop(df.columns[0], axis=1)
print(df.head())

         V1        V2        V3            V4        V5
0  0.040845  0.000049  0.000010  1.077239e-06  0.000244
1  0.000000  0.001735  0.000009  1.488535e-09  0.000000
2  0.257898  0.345134  0.281113  7.946676e-01  0.332110
3  0.819227  0.113165  0.147548  1.475484e-01  0.113165
4  0.026953  0.253826  0.044850  6.441857e-01  0.187666


## Apply the BU procedures with $\theta$ which give target Bonferroni power of $0.3$.

#### $p$-values to which we apply all the procedures

In [316]:
# Interesting Index = 9,20,21,24,32,34,45,54,84,92,102,124,130,142,166,170,189,200,232,233,246

In [318]:
for idx, pvals in df.iterrows():
    pvals[pvals == 0] = 1e-30
    print([idx,np.sum(decision_bayes(pvals,theta,th_bayes)),np.sum(GouConsonantRej(pvals,level=0.05)),np.sum(1*(multipletests(pvals, alpha=alpha, method='hommel')[1]<=0.05))])

[0, 5, 5, 5]
[1, 5, 5, 5]
[2, 0, 0, 0]
[3, 0, 0, 0]
[4, 0, 0, 0]
[5, 1, 1, 1]
[6, 0, 0, 0]
[7, 3, 3, 3]
[8, 3, 3, 3]
[9, 1, 0, 0]
[10, 1, 1, 1]
[11, 5, 5, 5]
[12, 0, 0, 0]
[13, 1, 1, 1]
[14, 0, 0, 0]
[15, 0, 0, 0]
[16, 0, 0, 0]
[17, 1, 1, 1]
[18, 3, 3, 3]
[19, 2, 2, 2]
[20, 1, 1, 0]
[21, 1, 1, 0]
[22, 0, 0, 0]
[23, 1, 1, 1]
[24, 2, 1, 0]
[25, 0, 0, 0]
[26, 0, 0, 0]
[27, 1, 1, 1]
[28, 0, 0, 0]
[29, 0, 0, 0]
[30, 0, 0, 0]
[31, 2, 2, 2]
[32, 2, 0, 0]
[33, 0, 0, 0]
[34, 4, 3, 3]
[35, 5, 5, 5]
[36, 0, 0, 0]
[37, 0, 0, 0]
[38, 0, 0, 0]
[39, 0, 0, 0]
[40, 0, 0, 0]
[41, 1, 1, 1]
[42, 5, 5, 5]
[43, 1, 1, 1]
[44, 3, 3, 3]
[45, 4, 2, 2]
[46, 4, 4, 4]
[47, 4, 4, 4]
[48, 5, 5, 5]
[49, 5, 5, 5]
[50, 3, 3, 3]
[51, 5, 5, 5]
[52, 2, 2, 2]
[53, 0, 0, 0]
[54, 3, 2, 2]
[55, 0, 0, 0]
[56, 5, 5, 5]
[57, 3, 3, 3]
[58, 1, 1, 1]
[59, 5, 5, 5]
[60, 4, 4, 4]
[61, 4, 4, 4]
[62, 5, 5, 5]
[63, 1, 1, 1]
[64, 5, 5, 5]
[65, 2, 2, 2]
[66, 2, 2, 2]
[67, 2, 2, 2]
[68, 5, 5, 5]
[69, 0, 0, 0]
[70, 0, 0, 0]
[71, 1, 1, 1]
[7

In [319]:
pvals = df.iloc[24,:]
pvals[pvals == 0] = 1e-30
print(np.array(pvals))

[0.02183311 0.17627716 0.31389191 0.01112552 0.19692412]


#### BU procedure with objective $\Pi_{mix}$

In [323]:
decision_bayes(pvals,theta,th_bayes)

array([1, 0, 0, 1, 0])

#### BU procedure with objective $\Pi_1$

In [326]:
decision_pi1(pvals,theta,th_pi1)

array([0, 0, 0, 0, 0])

#### Last step improvement of Hommel with objective $\Pi_{mix}$

In [329]:
decisions_hom_bayes(pvals,theta,thhom_bayes)

array([0, 0, 0, 1, 0])

#### Last step improvement of Hommel with objective $\Pi_1$

In [332]:
decisions_hom_pi1(pvals,theta,thhom_pi1)

array([0, 0, 0, 0, 0])

#### Hommel

In [335]:
1*(multipletests(pvals, alpha=alpha, method='hommel')[1]<=0.05)

array([0, 0, 0, 0, 0])

#### Gou

In [338]:
GouConsonantRej(pvals,level=0.05)

array([0, 0, 0, 1, 0])